This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [5]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'MyKey'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json
import quandl

In [13]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
quandl.ApiConfig.api_key = API_KEY
#Using quandl to request data
data = quandl.get('FSE/AFX_X',start_date='2001-01-01',end_date='2001-01-01')
data_dict = data.to_dict()
data_dict

{'Open': {Timestamp('2001-01-01 00:00:00'): None},
 'High': {Timestamp('2001-01-01 00:00:00'): None},
 'Low': {Timestamp('2001-01-01 00:00:00'): None},
 'Close': {Timestamp('2001-01-01 00:00:00'): 21.5},
 'Change': {Timestamp('2001-01-01 00:00:00'): None},
 'Traded Volume': {Timestamp('2001-01-01 00:00:00'): 0.0},
 'Turnover': {Timestamp('2001-01-01 00:00:00'): None},
 'Last Price of the Day': {Timestamp('2001-01-01 00:00:00'): None},
 'Daily Traded Units': {Timestamp('2001-01-01 00:00:00'): None},
 'Daily Turnover': {Timestamp('2001-01-01 00:00:00'): 0.0}}

In [30]:
#Using request to load data
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json'
r = requests.get(url)
new_data = r.json()


## Inspect the JSON structure of the object you created, and take note of how nested it is,
## as well as the overall structure

The data set has one key as 'dataset', and values is a dictionary,with the following keys:
    'id','dataset_code','database_code','name','description','refreshed_at','newest_available_date',
    'oldest_available_date','column_names','frequency',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2000-06-07',
  'end_date': '2019-08-02',
  'data':2d array

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [131]:
# 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, 
#for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

columns = new_data['dataset']['column_names']
entry = new_data['dataset']['data']
for i in range(0,len(entry)):
    if entry[i][0] == '2017-01-02':
        year_begin = i + 1
   
    if entry[i][0] == '2017-12-29':
        year_end = i

data2017 = new_data['dataset']['data'][year_end:year_begin]


In [132]:
# 2. Convert the returned JSON object into a Python dictionary.

dict_2017 = {}
for col in range(0,len(columns)):
    dict_2017[columns[col]] = [i[col] for i in data2017]
columns

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [96]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.
max_price = dict_2017['Open'][0]
min_price = dict_2017['Open'][0]
for price in dict_2017['Open']:
    if price and price > max_price:
        max_price = price
    if price and price < min_price:
        min_price = price
max_price, min_price

(53.11, 34.0)

In [100]:
# 4. What was the largest change in any one day (based on High and Low price)?
max_change = dict_2017['High'][0] - dict_2017['Low'][0]
for (high,low) in zip(dict_2017['High'],dict_2017['Low']):
    if high and low:
        change = high - low
    if change and price > max_change:
        max_change = change
max_change

0.9499999999999957

In [102]:
# 5. What was the largest change between any two days (based on Closing Price)?
max_two_day_change = dict_2017['Close'][0]
for change in dict_2017['Close']:
    if change and change > max_two_day_change:
        max_two_day_change = change
max_two_day_change

53.09

In [108]:
# 6. What was the average daily trading volume during this year?

average = sum(dict_2017['Traded Volume'])/len(dict_2017['Traded Volume'])
average

89124.33725490196

In [128]:
# 7. What was the median trading volume during this year. 
# (Note: you may need to implement your own function for calculating the median.)
def medium(array):
    length = len(array)
    if length % 2 == 0:
        return (array[int(length/2)-1] + array[int(length/2)])/2
    else:
        return array[int(length/2)]
    
array = sorted(dict_2017['Traded Volume'])

medium(array)

76286.0